In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
.master("local[4]") \
.appName("IrisClassification") \
.config("spark.executor.memory", "1g") \
.config("spark.driver.memory", "1g") \
.getOrCreate()

In [ ]:
iris = spark.read \
.option("header", True) \
.option("inferSchema", True) \
.option("sep", ",") \
.csv("/home/alper/Spark/data/iris.csv")

In [ ]:
iris.limit(5).toPandas().head()

In [ ]:
iris.count()

In [ ]:
iris.printSchema()

In [ ]:
numeric = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
categorical = ["species"]

In [ ]:
iris.describe(numeric).toPandas()

In [ ]:
from pyspark.sql import functions as F

In [ ]:
iris.groupBy(F.col("species")).agg({"*": "count"}).toPandas()

In [ ]:
null_count = 1
for column in iris.columns:
    if (iris.filter(F.col(column).isNull()).count() > 0):
        print(null_count, ". ", column, " has null.")
    else:
        print(null_count, ". ", column)
        
    null_count += 1

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler

In [ ]:
species_indexer = StringIndexer() \
.setInputCol("species") \
.setOutputCol("label")

In [ ]:
iris = species_indexer.fit(iris).transform(iris)

In [ ]:
iris.toPandas().head()

In [ ]:
assembler = VectorAssembler() \
.setInputCols(numeric) \
.setOutputCol("features")

In [ ]:
iris = assembler.transform(iris)

In [ ]:
iris.toPandas().head()

In [ ]:
train_df, test_df = iris.randomSplit([0.8, 0.2], seed=4242)

In [ ]:
print("Train: ", train_df.count())
print("Test: ", test_df.count())

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
logreg_object = LogisticRegression() \
.setLabelCol("label") \
.setFeaturesCol("features")

In [ ]:
model = logreg_object.fit(train_df)

In [ ]:
transformedDF = model.transform(test_df)

In [ ]:
transformedDF.limit(5).toPandas().head()

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
multi = MulticlassClassificationEvaluator() \
.setLabelCol("label") \
.setPredictionCol("prediction") \
.setMetricName("accuracy")

In [ ]:
accuracy = multi.evaluate(transformedDF)

In [ ]:
accuracy